# Al, Mg and Be results

In [1]:
import os, json
import numpy as np

from monty.json import MontyDecoder, MontyEncoder
from quotas.core import QuotasCalculator

data_dir = "/Users/mbercx/Dropbox/Phd/thesis/jupyter/quotas/data"
tables_dir = "/Users/mbercx/Dropbox/Phd/thesis/tables/quotas"

# Load the experimental data
with open(os.path.join(data_dir, "experimental_data.json"), "r") as file:
    exp_data = json.loads(file.read(), cls=MontyDecoder)
    
# Define the effective ionization energies of the rare gas ions
ionization_energies = {
    "He": 24.56 - 2,
    "Ne": 21.56 - 1,
    "Ar": 15.76 - 0.5,
    "Kr": 12.00,
    "Xe": 10.13
}

exp_work_function = {
    "Al": {
        "111": 4.28,
        "100": 4.36,
        "110": 4.21
    },
    "Mg": {
        "100": 3.66,
        "001": 3.66,
        "110": 3.66
    },
    "Ni": {
        "111": 5.28,
        "100": 5.23,
        "110": 4.64
    },
    "Cu": {
        "110": 4.48,
        "111": 4.91,
        "100": 4.57
    },
    "Be": {
        "100": 4.98,
        "001": 4.98,
        "110": 4.98
    },
    "W": {
        "111": 4.38,
        "100": 4.57,
        "110": 5.31
    }
}

In [2]:
data_array = []

for element in exp_work_function.keys():
    
    for i, surface in enumerate(exp_work_function[element].keys()):
        
        quotas = QuotasCalculator.from_file(
            os.path.join(data_dir, element + "_" + surface + "_pbe.json")
        )
        
        row = []
        if i == 0 and element is not "Al":
            row.append(r"\hline")
        elif i == 1:
            row.append(element)
        else:
            row.append("")
            
        row.append("(" + surface + ")")
        row.append(round(quotas.workfunction_data.work_function, 2),)
        if i == 0 or element in ["Al", "W", "Cu", "Ni"]:
            row.append(exp_work_function[element][surface])
        else:
            row.append(r'"')
        
        see_He = quotas.calculate_yield(ionization_energies["He"])
        row.append(round(see_He["total_yield"], 3))
        exp_yield_He = np.trapz(
            exp_data[element]["He"]["yield"], exp_data[element]["He"]["energy"]
        ) / 1e3
        if i == 0:
            row.append(round(exp_yield_He, 3))
        elif element in ["Ni", "Cu"]:
            row.append(r'-')
        else:
            row.append(r'"')
        
        see_Ne = quotas.calculate_yield(ionization_energies["Ne"])
        row.append(round(see_Ne["total_yield"], 3))
        exp_yield_Ne = np.trapz(
            exp_data[element]["Ne"]["yield"], exp_data[element]["Ne"]["energy"]
        ) / 1e3
        if i == 0:
            row.append(round(exp_yield_Ne, 3))
        elif element in ["Ni", "Cu"]:
            row.append(r'-')
        else:
            row.append(r'"')
        
        data_array.append(row)

In [3]:
data_array

[['', '(111)', 4.05, 4.28, 0.193, 0.231, 0.157, 0.202],
 ['Al', '(100)', 4.26, 4.36, 0.175, '"', 0.141, '"'],
 ['', '(110)', 4.04, 4.21, 0.195, '"', 0.159, '"'],
 ['\\hline', '(100)', 3.65, 3.66, 0.286, 0.257, 0.222, 0.202],
 ['Mg', '(001)', 3.7, '"', 0.242, '"', 0.195, '"'],
 ['', '(110)', 3.49, '"', 0.316, '"', 0.245, '"'],
 ['\\hline', '(111)', 5.05, 5.28, 0.173, 0.175, 0.142, 0.124],
 ['Ni', '(100)', 4.91, 5.23, 0.181, '-', 0.15, '-'],
 ['', '(110)', 4.67, 4.64, 0.203, '-', 0.171, '-'],
 ['\\hline', '(110)', 4.4, 4.48, 0.145, 0.155, 0.12, 0.12],
 ['Cu', '(111)', 4.77, 4.91, 0.125, '-', 0.101, '-'],
 ['', '(100)', 4.52, 4.57, 0.139, '-', 0.114, '-'],
 ['\\hline', '(100)', 4.5, 4.98, 0.083, 0.117, 0.054, 0.095],
 ['Be', '(001)', 5.31, '"', 0.049, '"', 0.029, '"'],
 ['', '(110)', 3.82, '"', 0.119, '"', 0.081, '"'],
 ['\\hline', '(111)', 4.15, 4.38, 0.161, 0.292, 0.124, 0.224],
 ['W', '(100)', 4.1, 4.57, 0.158, '"', 0.121, '"'],
 ['', '(110)', 4.79, 5.31, 0.117, '"', 0.082, '"']]

In [4]:
np.savetxt(os.path.join(tables_dir, "metal_results.tex"), 
           data_array, 
           delimiter=' & ', 
           fmt='%s',#['%s', '%s', '%2.2e', '%2.2e', '%2.2e', '%2.2e', '%2.2e', '%2.2e'], 
           newline=' \\\\\n')
